In [1]:
import torch
from PIL import Image
import torchvision.transforms as T
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import os
import argparse
import json
from tqdm import tqdm

In [2]:
def extract_features(img_type, input_image):
    if img_type == "vit":
        config = resolve_data_config({}, model=vit_model, verbose=1)
        transform = create_transform(**config)
        with torch.no_grad():
            img = Image.open(input_image).convert("RGB")
            input = transform(img).unsqueeze(0)
            feature = vit_model.forward_features(input)
        return feature
    
    elif img_type == "detr":
        transform = T.Compose([
            T.Resize(224),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        with torch.no_grad():
            img = Image.open(input_image).convert("RGB")
            input = transform(img).unsqueeze(0)
            feature = detr_model(input)[-1]
        return feature

In [3]:
data_root = 'data/images'
all_images = os.listdir(data_root)
all_images

['1',
 '100',
 '10001',
 '10003',
 '10007',
 '10008',
 '10009',
 '1001',
 '10010',
 '10014',
 '10015',
 '10017',
 '10019',
 '1002',
 '10023',
 '10024',
 '10025',
 '10026',
 '10027',
 '1003',
 '10032',
 '10038',
 '10039',
 '10044',
 '10046',
 '10053',
 '10055',
 '10056',
 '10057',
 '10059',
 '10066',
 '10067',
 '10069',
 '10071',
 '10072',
 '10073',
 '10076',
 '10077',
 '10078',
 '10079',
 '10080',
 '10081',
 '10082',
 '10084',
 '10085',
 '10087',
 '10088',
 '1009',
 '10090',
 '10092',
 '10094',
 '10095',
 '10098',
 '10099',
 '101',
 '1010',
 '10100',
 '10105',
 '10107',
 '1011',
 '10111',
 '10116',
 '10117',
 '10118',
 '1012',
 '10123',
 '10126',
 '10128',
 '1013',
 '10130',
 '10132',
 '10134',
 '10135',
 '10139',
 '1014',
 '10141',
 '10143',
 '10145',
 '10147',
 '10148',
 '1015',
 '10150',
 '10152',
 '10157',
 '10159',
 '10161',
 '10162',
 '10163',
 '10168',
 '10169',
 '1017',
 '10171',
 '10172',
 '10176',
 '10177',
 '10178',
 '10179',
 '1018',
 '10180',
 '10181',
 '10182',
 '10183',


In [4]:
tmp = []
name_map = {}
all_images.sort(key=lambda x:int(x))
print(len(all_images))

11208


In [16]:
vit_model = timm.create_model("vit_large_patch32_384", pretrained=True, num_classes=0)
vit_model.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(32, 32), stride=(32, 32))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Id

In [17]:
for idx, image in enumerate(tqdm(all_images)):
    if idx % 100 == 0: print(idx)
    if os.path.exists(os.path.join(data_root, image, "image.png")):
        curr_dir = os.path.join(data_root, image, "image.png")
        feature = extract_features('vit', curr_dir)
        tmp.append(feature.detach().cpu())
        name_map[str(image)] = idx

  0%|                                                                                        | 0/11208 [00:00<?, ?it/s]

0


  1%|▋                                                                           | 100/11208 [01:23<2:48:37,  1.10it/s]

100


  2%|█▎                                                                          | 200/11208 [02:38<2:16:04,  1.35it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████▎| 11100/11208 [4:05:58<01:33,  1.16it/s]

11100


100%|█████████████████████████████████████████████████████████████████████████▉| 11200/11208 [4:07:32<00:07,  1.11it/s]

11200


100%|██████████████████████████████████████████████████████████████████████████| 11208/11208 [4:07:40<00:00,  1.33s/it]


In [18]:
res = torch.cat(tmp).cpu()
torch.save(res, os.path.join('vision_features', 'vit_large_patch32_384' +'.pth'))
with open(os.path.join('vision_features', 'vit_large_patch32_384_name_map.json'), 'w') as outfile:
    json.dump(name_map, outfile)

In [14]:
detr_model = torch.hub.load('cooelf/detr', 'detr_resnet101_dc5', pretrained=True)
detr_model.eval()

C:\Users\pakale\Anaconda3\lib\site-packages\torch\hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/cooelf/detr/zipball/main" to C:\Users\pakale/.cache\torch\hub\main.zip
C:\Users\pakale\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\pakale\Anacon

DETR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, ou

In [15]:
for idx, image in enumerate(tqdm(all_images)):
    if idx % 100 == 0: print(idx)
    if os.path.exists(os.path.join(data_root, image, "image.png")):
        curr_dir = os.path.join(data_root, image, "image.png")
        feature = extract_features('detr', curr_dir)
        tmp.append(feature.detach().cpu())
        name_map[str(image)] = idx

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

0


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.18it/s]


In [16]:
res = torch.cat(tmp).cpu()
print(res.shape)
torch.save(res, os.path.join('vision_features', 'detr_resnet101_dc5' +'.pth'))
with open(os.path.join('vision_features', 'detr_resnet101_dc5_name_map.json'), 'w') as outfile:
    json.dump(name_map, outfile)

torch.Size([4, 145, 1024])


In [ ]:
processor = AutoImageProcessor.from_pretrained("SenseTime/deformable-detr-with-box-refine-two-stage")
model = DeformableDetrForObjectDetection.from_pretrained("SenseTime/deformable-detr-with-box-refine-two-stage")

In [ ]:
for idx, image in enumerate(tqdm(all_images)):
    if idx % 100 == 0: print(idx)
    if os.path.exists(os.path.join(data_root, image, "image.png")):
        input_image = os.path.join(data_root, image, "image.png")
        img = Image.open(input_image).convert("RGB")
        inputs = processor(images=img, return_tensors="pt")
        outputs = model(**inputs)
        tmp.append(outputs['last_hidden_state'].detach().cpu())
        name_map[str(image)] = idx

In [ ]:
res = torch.cat(tmp).cpu()
print(res.shape)
torch.save(res, os.path.join('vision_features', 'deformable-detr-with-box-refine-two-stage' +'.pth'))
with open(os.path.join('vision_features', 'deformable-detr-with-box-refine-two-stage_name_map.json'), 'w') as outfile:
    json.dump(name_map, outfile)